# video-translation-service（Colab + T4 GPU）一键安装运行

目标：上传 `1.mp4` → 生成中文字幕 `1_zh.srt`（不启用润色）。

注意：首次运行会下载 Whisper + 翻译模型（可能几 GB），需要等待一段时间。

In [ ]:
#@title 0) 确认已启用 GPU（Runtime -> Change runtime type -> GPU -> T4）
!nvidia-smi -L

import sys
import torch

print("python:", sys.version)
print("torch:", torch.__version__)
print("cuda_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
else:
    raise RuntimeError("未检测到GPU：请在 Colab 切换到 GPU(T4) 运行时")


In [ ]:
#@title 1) git clone 项目
# TODO: 改成你的仓库地址（私有仓库可用：https://<TOKEN>@github.com/<org>/<repo>.git）
REPO_URL = "https://github.com/wujelly701/video-translation-service.git"
BRANCH = "main"
REPO_DIR = "/content/video-translation-service"

import os
import shutil

if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone --depth 1 -b {BRANCH} {REPO_URL} {REPO_DIR}
%cd {REPO_DIR}


In [ ]:
#@title 2) 安装系统依赖（ffmpeg）
!apt-get update -y
!apt-get install -y ffmpeg
!ffmpeg -version | head -n 2


In [ ]:
#@title 3) 安装 Python 依赖
# 说明：Colab 自带 CUDA 版 torch，避免从 requirements.txt 里重复安装 torch（否则可能被换成CPU版/或耗时升级）
!python -m pip install -U pip
!grep -vE '^torch' requirements.txt > /tmp/requirements_no_torch.txt
!python -m pip install -r /tmp/requirements_no_torch.txt

# 可选：人声分离（Demucs）用于嘈杂/背景音乐场景
!python -m pip install demucs


In [ ]:
#@title 4) 生成 config.ini（GPU + 中文 + 不润色）
from pathlib import Path

config_text = """[API]
deepseek_api_key =

[Service]
host = 127.0.0.1
port = 50515

[Models]
# 如果显存不够/加载太慢，可改：asr_model_size = small
asr_model_size = medium

# T4 显存有限，默认使用 600M（更稳）；追求质量可改为 1.3B
translation_model = facebook/nllb-200-distilled-600M

use_gpu = true
beam_size = 3

[Translation]
default_target_language = zh
use_deepseek_polish = false

[Audio]
# 人声分离（Demucs）：改善背景音乐/嘈杂场景识别
enable_vocal_separation = true
vocal_separation_model = htdemucs
# 建议先用 cpu，避免与服务端模型争抢显存导致 OOM
vocal_separation_device = cpu
"""

Path("config.ini").write_text(config_text, encoding="utf-8")
print("wrote config.ini")


In [ ]:
#@title 5) 后台启动服务（不占用单元格）
import pathlib
import subprocess
import sys

import requests

SERVICE_URL = "http://127.0.0.1:50515"

def health():
    try:
        return requests.get(f"{SERVICE_URL}/health", timeout=2).json()
    except Exception:
        return None

h = health()
if not h:
    p = subprocess.Popen(
        [sys.executable, "server_optimized.py"],
        stdout=open("server.log", "wb"),
        stderr=subprocess.STDOUT,
        start_new_session=True,
    )
    pathlib.Path("server.pid").write_text(str(p.pid))
    print("server started, pid:", p.pid)
else:
    print("server already running:", h)


In [ ]:
#@title 6) 等待模型加载完成（首次运行需要等待）
import json
import os
import time
from pathlib import Path

import requests

SERVICE_URL = "http://127.0.0.1:50515"
pid_path = Path("server.pid")
log_path = Path("server.log")

def pid_alive(pid: int) -> bool:
    try:
        os.kill(pid, 0)
        return True
    except Exception:
        return False

def tail_log(max_lines: int = 120) -> str:
    if not log_path.exists():
        return "(server.log not found)"
    try:
        lines = log_path.read_text(errors="ignore").splitlines()
        return "\n".join(lines[-max_lines:])
    except Exception as e:
        return f"(failed to read server.log: {e})"

for i in range(1800):  # 最多等 3600 秒
    try:
        h = requests.get(f"{SERVICE_URL}/health", timeout=5).json()
    except Exception as e:
        if i % 5 == 0:
            print(f"{i*2:>4}s", "waiting for server...", repr(e))
        if pid_path.exists():
            try:
                pid = int(pid_path.read_text().strip() or "0")
            except Exception:
                pid = 0
            if pid and not pid_alive(pid):
                print("\nserver process exited; last logs:\n")
                print(tail_log())
                raise
        time.sleep(2)
        continue

    if h.get("ready"):
        print("READY:\n", json.dumps(h, ensure_ascii=False, indent=2))
        break
    if i % 5 == 0:
        print(f"{i*2:>4}s", h.get("phase"), h.get("progress"), h.get("message"))
    if h.get("phase") == "error":
        raise RuntimeError(h.get("error") or "模型加载失败，请查看 server.log")
    time.sleep(2)
else:
    raise TimeoutError("等待服务就绪超时：请查看 server.log")


In [ ]:
#@title 7) 上传测试视频（请上传 1.mp4）
from google.colab import files

uploaded = files.upload()
print("uploaded:", list(uploaded.keys()))

# 如果你上传的文件名不是 1.mp4，但只上传了1个文件，则自动重命名
import os

if "1.mp4" not in uploaded and len(uploaded) == 1:
    src = list(uploaded.keys())[0]
    os.rename(src, "1.mp4")
    print("renamed:", src, "-> 1.mp4")


In [ ]:
#@title 8) 翻译生成中文字幕（不润色）
!python batch_translate.py 1.mp4 -t zh --translation-only


In [ ]:
#@title 9) 下载生成的字幕文件
from google.colab import files

files.download("1_zh.srt")


In [ ]:
#@title 10) （可选）停止服务
import os
import signal
import pathlib

pid_path = pathlib.Path("server.pid")
if pid_path.exists():
    pid = int(pid_path.read_text().strip())
    os.kill(pid, signal.SIGTERM)
    print("killed:", pid)
else:
    print("server.pid not found")
